# What's new in scipp 0.6.0 and scippneutron 0.1.0

## `scipp.neutron` is now `scippneutron`

- [Release Notes](https://scipp.github.io/about/release-notes.html#v0-6-march-2021) for scipp
- [Release Notes](https://scipp.github.io/scippneutron/about/release-notes.html#v0-1-march-2021) for scippneutron

As part of a process to ensure that scipp is inviting and open to external collaborators we have removed `scipp.neutron`.
The neutron-scattering-specific functionality is now provided by a new and separate package, [scippneutron](https://scipp.github.io/scippneutron/).

Download [PG3_4844_event.nxs](http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e) (and rename after download) to run this notebook.

In [ ]:
import numpy as np
import scipp as sc
if not sc.__version__.startswith('0.6'):
    print(f'This notebook was made for scipp-0.6 and will likely not work with your version ({sc.__version__}).')

## scipp-0.6.0

### Support for datetime64

- Previously we stored time-related information such as `pulse_time` and times from, e.g., sample-temperature logs as integers.
- Added support for datetime64 compatible with [np.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html)
- Time differences (`np.timedelta64`) are not used, we simply use integers since in combination with scipp's units this provides everything we need.

Example:

In [ ]:
var = sc.array(dims=['time'], values=np.arange(
    np.datetime64('2021-01-01T12:00:00'),
    np.datetime64('2021-01-01T12:04:00')))
var + 123 * sc.Unit('s')

Data loaded from Nexus files now uses `datetime64` for `pulse_times` and time-series logs such as sample temperature logs:

In [ ]:
import scippneutron as scn
if not scn.__version__.startswith('0.1'):
    print(f'This notebook was made for scippneutron-0.1 and will likely not work with your version ({scn.__version__}).')
filename = 'PG3_4844_event'
data = scn.load(filename=f'{filename}.nxs', mantid_args={'LoadMonitors':True})

In [ ]:
sc.table(data.attrs['ChopperStatus1'].value)

In [ ]:
sc.table(data.bins.constituents['data']['event', :5])

### HTML view readability improvements

The readability for nested scipp objects such as `DataArray` in the HTML view has been improved, see in particular the many scalar attributes:

In [ ]:
data

### SVG view readability improvements

The SVG view now indicates binned (event) data.
Note that the number of drawn dots is not related to the actual number of events:

In [ ]:
sc.show(data)

### Unit conversions

Conversions between different unit scales (not to be confused with [conversions provided by scippneutron](https://scipp.github.io/scippneutron/user-guide/unit-conversions.html)) are now supported:

In [ ]:
var = 1.2 * sc.Unit('m')
sc.to_unit(var, unit='mm')

In [ ]:
sc.to_unit(sc.scalar(1.0, unit='nJ'), unit='meV')

### `fold` and `flatten`

`fold` and `flatten`, which are similar to `reshape`, have been added.
In contrast to `reshape`, `fold` and `flatten` support data arrays and handle also meta data such as coord, masks, and attrs:

In [ ]:
var = sc.ones(dims=['pixel'], shape=[100])
xy = sc.fold(var, dim='pixel', sizes={'x':10, 'y':10})
xy = sc.DataArray(data=xy,
                  coords={
                      'x':sc.array(dims=['x'], values=np.arange(10)),
                      'y':sc.array(dims=['y'], values=np.arange(10))})
xy

In [ ]:
sc.flatten(xy, to='pixel')

### Binned data meta data access

The internal buffer and indices defining binned data are now accessible in the `data.bins.constituents` dictionary.
For example, the flat buffer holding event data can be accessed as:

In [ ]:
data.bins.constituents['data']

This could be used, e.g., to access the `tof` coordinate of the events.
However, this does not give information on which bin (pixel in this case) an event belongs to, so this is not sufficient if, e.g., a computation involving beamline geometry should be performed.

To allow for this, we added support for access to the coords (and data, masks, and attrs) of the bins while retaining the binned structure.
The `bins` property now provides properties `data`, `coords`, `masks`, and `attrs` that behave like the properties of a data array.
We can use this, e.g., to compute the time an event passed the sample, and store it as a new coord:

In [ ]:
Ltotal = scn.Ltotal(data, scatter=True)
L2 = scn.L2(data)
data.bins.coords['time_at_sample'] = data.bins.coords['tof'] * (1 - L2/Ltotal) # broadcast pixel-dependent factor to all events in a bin
sc.table(data.values[10000]['event', :5])

Note that at this point things such as the HTML representation are not supported for these bin components.

## scippneutron-0.1.0

### Various

- As a small breaking change, the naming of metadata was changed to use `_` instead of `-`, e.g., `pulse_time`, `sample_position`, and `incident_energy`.
- The contents of (previously) `scipp.neutron.diffraction` are not available any more, since they were considered as too specialized (facility specific).
  For `scipp.neutron.diffraction.convert_with_calibration` see [Beamline geometry parameters for unit conversions](Beamline-geometry-parameters-for-unit-conversions) below for the first iteration of a better approach which could be extented to support something similar in a more flexible manner.
- `from_mantid` (and by extension `load`) have been improved to support loading of `McStasNexus` files and `WorkspaceGroup` workspaces.
- `convert` now supports direct-inelastic and indirect-ineleastic conversions to `energy_transfer`.
  A coord named `incident_energy` or `final_energy` is required in this case.
  Where possible `from_mantid` (and by extension `load`) attempt to extract this information from the workspace, but in many cases this is not possible or available, so these must be set manually.

### Mandatory `scatter` argument for unit conversion

- Previously `scn.convert` "guessed" whether unit conversions should assume scattering from a sample based on the presence of a coord or attr named `sample_position`.
  To correctly convert, e.g., to wavelength for a monitor we had to ensure that `sample_position` was not present.
- This brittle and error-prone mechanism has been replaced by a *mandatory* new parameter name `scatter` to `convert`.

Example:

In [ ]:
sc.to_html(scn.convert(data, 'tof', 'wavelength', scatter=True))
sc.to_html(scn.convert(data.attrs['monitor1'].value, 'tof', 'wavelength', scatter=True)) # scatter from sample into monitor?
sc.to_html(scn.convert(data.attrs['monitor1'].value, 'tof', 'wavelength', scatter=False))

### Beamline geometry parameters for unit conversions

Unit conversions are now handling beamline geometry in a more flexible manner.
Please see [the documentation](https://scipp.github.io/scippneutron/user-guide/unit-conversions.html#Beamline-geometry-parameters-used-in-unit-conversions) for a graphical illustration of the mechanism.

- Opens opportunities for more flexible definition of beamline geometry.
- Should make it easier to handle cases such as multiple indicent angles or indirect-geometry beamlines.
- Allows for overriding parameters such as positions with, e.g., calibrated $L_2$.

Example:

In [ ]:
sc.to_html(scn.convert(data, 'tof', 'wavelength', scatter=True)) # using `position` to compute L2
data.coords['L2'] = 1.0 * sc.Unit('m')
sc.to_html(scn.convert(data, 'tof', 'wavelength', scatter=True)) # using provided L2

In above example we used a scalar `L2` coord.
This is then broadcast to all pixels and thus implies that all pixels are effectively on a sphere around the sample.
Use, e.g., `L2` dependent on the `spectrum` dimension to set a pixel-dependent `L2`.